In [ ]:
import numpy as np  
import pandas as pd 
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import plotly.express as px
from urllib.request import urlopen
from pandas.io.json import json_normalize
import json
import requests 
from shapely.geometry import shape, Point
import seaborn as sns
import matplotlib.pyplot as plt
from folium import IFrame
from google.colab import files
from time import sleep

In [ ]:
# Load Taipei GeoJSON file

with urlopen('https://raw.githubusercontent.com/littlebtc/geodata4appy/master/simplified/taipei.geojson') as response:
    tp = json.load(response)

In [ ]:
# Check on map 
address = 'Taipei'

geolocator = Nominatim(user_agent="tp_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

tp_map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.GeoJson(
    tp,
    name='geojson'
).add_to(tp_map)

tp_map

In [ ]:
# Check in dataframe
df_tp = json_normalize(tp['features'])

# Rename column names for later use
df_tp.rename(columns={'properties.TVNAME':'dist_neigh', 
                      'properties.PNAME':'city',
                      'properties.TNAME':'dist',
                      'properties.VNAME':'neigh'
                      }, 
             inplace=True)

df_tp.tail()

In [ ]:
# Functions to find the centroid of a polygon in GeoJSON file
# Credited to brandonxiang/geojson-python-utils: https://github.com/brandonxiang/geojson-python-utils/blob/develop/README_CN.md

def centroid(poly):
    """
    get the centroid of polygon
    adapted from http://paulbourke.net/geometry/polyarea/javascript.txt
    Keyword arguments:
    poly -- polygon geojson object
    return polygon centroid
    """
    f_total = 0
    x_total = 0
    y_total = 0
    # TODO: polygon holes at coordinates[1]
    points = poly['coordinates'][0]
    j = len(points) - 1
    count = len(points)

    for i in range(0, count):
        p1_x = points[i][1]
        p1_y = points[i][0]
        p2_x = points[j][1]
        p2_y = points[j][0]

        f_total = p1_x * p2_y - p2_x * p1_y
        x_total += (p1_x + p2_x) * f_total
        y_total += (p1_y + p2_y) * f_total
        j = i

    six_area = area(poly) * 6
    return {'type': 'Point', 'coordinates': [y_total / six_area, x_total / six_area]}

def area(poly):
    """
    calculate the area of polygon
    Keyword arguments:
    poly -- polygon geojson object
    return polygon area
    """
    poly_area = 0
    # TODO: polygon holes at coordinates[1]
    points = poly['coordinates'][0]
    j = len(points) - 1
    count = len(points)

    for i in range(0, count):
        p1_x = points[i][1]
        p1_y = points[i][0]
        p2_x = points[j][1]
        p2_y = points[j][0]

        poly_area += p1_x * p2_y
        poly_area -= p1_y * p2_x
        j = i

    poly_area /= 2
    return poly_area

In [ ]:
# Find centroid of every neighborhood and add the laitudes and longitudes to new columns 

tp_lon = []
tp_lat = []

for i in range(456):
  box = tp['features'][i]['geometry']
  center = centroid(box)
  tp_lon.append(center['coordinates'][0])
  tp_lat.append(center['coordinates'][1])

df_tp['Latitude'] = tp_lat
df_tp['Longitude'] = tp_lon

df_tp.tail()

In [ ]:
# Solve encoding problem, credited to: https://yafei777.github.io/Yafei-blog/blog/folium-popup/
def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

In [ ]:
# Check the covering area of 1500m-radius circles 

paragraph = """<p>{}</p>""".format

for lat, lng, name in zip(df_tp['Latitude'], df_tp['Longitude'], df_tp['dist_neigh']):
  label = '{}'.format(name)
  iframe = IFrame(html = paragraph(utf2asc(label)), width = 200, height = 100)
  popup = folium.Popup(iframe) 
  folium.Circle(
      [lat, lng], 
      popup=popup, 
      fill_color='#00FFFFFF', 
      radius=1500, 
      weight=2, 
      color="#000").add_to(tp_map)

tp_map

In [ ]:
# Add labels for each neighborhoods 
def add_label():
  paragraph = """<p>{}</p>""".format

  for lat, lng, dist, neigh in zip(df_tp['Latitude'], df_tp['Longitude'], df_tp['dist'], df_tp['neigh']):
    label = '{}, {}'.format(dist, neigh)
    iframe = IFrame(html = paragraph(utf2asc(label)),
                width=100,
                height=50)
    popup = folium.Popup(iframe)
    folium.CircleMarker(
        [lat, lng],
        radius = 3,
        popup = popup,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map)  

  return map

In [ ]:
# Read Taipei household income data
# Source: https://ws.fia.gov.tw/001/upload/ias/isa106/106_165-A.html 106年資料

income = pd.read_excel('https://github.com/dubidub/Coursera_Capstone/raw/master/Taiwan%20neighborhood%20income%202017.xlsx')
income.rename(
  columns={'行政區':'dist', '村里':'neigh', '中位數':'income_median'},
  inplace=True
)

income.sort_values('income_median', ascending=False)

In [ ]:
# Merge income with df_tp

# Create the merge key for both df
income["dist_neigh"] = income["dist"] + income["neigh"]

# Then merge 
df_tp1 = pd.merge(df_tp, income, on = 'dist_neigh')

In [ ]:
df_tp1.head()

In [ ]:
# Read Taiwan neighborhoods population data
# Source: https://ca.gov.taipei/News_Content.aspx?n=8693DC9620A1AABF&sms=D19E9582624D83CB&s=78DC4B104D9D374E

ppl = pd.read_excel('https://github.com/dubidub/Coursera_Capstone/raw/master/Taipei%20Neighborhoods%20population%202019.xlsx')


# Sum up populaion under 15
ppl.rename(columns={'行政區':'dist', '村里':'neigh',  
                    '0歲':'m0', '1歲':'m1', '2歲':'m2', '3歲':'m3', '4歲':'m4',
                    '5歲':'m5', '6歲':'m6', '7歲':'m7', '8歲':'m8', '9歲':'m9', 
                    '10歲':'m10', '11歲':'m11', '12歲':'m12',
                    '13歲':'m13', '14歲':'m14', '15歲':'m15',
                    },
              inplace=True)

ppl['under_15'] = ppl.m0 + ppl.m1 + ppl.m2 + ppl.m3 + ppl.m4 + ppl.m5 + ppl.m6 \
                  + ppl.m7 + ppl.m8 + ppl.m9 + ppl.m10 + ppl.m11 + ppl.m12 + \
                  ppl.m13 + ppl.m14 + ppl.m15 

ppl.sort_values('under_15', ascending=False)

In [ ]:
# Merge ppl with df_tp1

# Create the merge key for both df
ppl["dist_neigh"] = ppl["dist"] + ppl["neigh"]

# Then merge 
df_tp2 = pd.merge(df_tp1, ppl, on = 'dist_neigh')

df_tp2.head()

In [ ]:
df_tp3 = df_tp2.loc[:, ['dist_neigh', 'Latitude', 'Longitude', 'under_15', 'income_median']]
df_tp3.rename(columns={'properties.TVNAME':'dist_neigh', 
                      'properties.PNAME':'city',
                      'properties.TNAME':'dist',
                      'properties.VNAME':'neigh'
                      }, 
             inplace=True)

In [ ]:
# Find the missing values
missing = pd.merge(df_tp3, df_tp, on='dist_neigh', how='outer')

missing_dist = missing['dist_neigh'].loc[missing['Latitude_x'].isnull()]
missing_dist = list(missing_dist)
for i in missing_dist:
  print(i)

In [ ]:
missing['missing'] = missing['Latitude_x'].isnull().astype(int)
missing

In [ ]:
# Show missing neighborhoods on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'Missing neighborhoods',
    data = missing,
    columns = ['dist_neigh', 'missing'],
    key_on = 'properties.TVNAME',
    fill_color = 'YlGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Missing Neighborhoods').add_to(map)

# add_label()

map

In [ ]:
df_tp3['under_15'].describe()

In [ ]:
df_tp3['income_median'].describe()

In [ ]:
fig, axs = plt.subplots(ncols=2)
fig.set_figwidth(15)
sns.distplot(df_tp3['under_15'], ax=axs[0])
sns.distplot(df_tp3['income_median'], ax=axs[1])

In [ ]:
# Show income on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'Taipei Household income',
    data = df_tp3,
    columns = ['dist_neigh', 'income_median'],
    key_on = 'properties.TVNAME',
    fill_color = 'YlGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Household Income').add_to(map)

# add_label()

map

In [ ]:
# Show ppl on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'under 15 population',
    data = df_tp3,
    columns = ['dist_neigh', 'income_median'],
    key_on = 'properties.TVNAME',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Population').add_to(map)

# add_label()

map

# Use Places API to get accessible education resources

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, placesTypes, radius, API_KEY):

...

    return(nearby_venues)

In [ ]:
API_KEY = 'YOUR_API_KEY'
placesTypes = ['primary_school', 'secondary_school', 'library', 'museum']
radius = 1500

In [ ]:
tp_venues = getNearbyVenues(df_tp3['dist_neigh'],
                            df_tp3['Latitude'],
                            df_tp3['Longitude'],
                            placesTypes,
                            radius,
                            API_KEY
                            )

In [ ]:
# Check the numbers of accessible education resources of each neighborhood

tp_edu = tp_venues.groupby('Neighbourhood')['v_cat'].value_counts().unstack(fill_value=0)
tp_edu = tp_edu.reset_index()
tp_edu['school'] = tp_edu['primary_school'] + tp_edu['secondary_school']
tp_edu['edu_Total'] = tp_edu['library'] + tp_edu['museum'] + tp_edu['school'] 
tp_edu.rename(columns={'library':'Library', 
                      'museum':'Museum',
                      'school':'School'
                      }, 
              inplace=True)
tp_edu.drop(columns=['primary_school', 'secondary_school'], inplace=True)
tp_edu.sort_values('edu_Total', ascending=False)

v_cat,Neighbourhood,Library,Museum,School,edu_Total
29,中山區民安里,42,59,29,130
27,中山區正得里,44,58,22,124
44,中正區南福里,43,44,36,123
53,中正區文北里,47,50,26,123
52,中正區愛國里,42,45,32,119
...,...,...,...,...,...
197,南港區九如里,0,2,0,2
157,北投區八仙里,0,2,0,2
115,內湖區內溝里,0,1,0,1
257,士林區菁山里,0,1,0,1


In [ ]:
# Find the missing values
tp_edu1 = pd.merge(df_tp3, tp_edu, left_on = 'dist_neigh', right_on='Neighbourhood', how='outer')
tp_edu1.drop('Neighbourhood', axis=1, inplace=True)
tp_edu1.fillna(0, inplace=True)
tp_edu1.sort_values('edu_Total', ascending=False)

In [ ]:
# Check unique venues
tp_venues2 = tp_venues.drop(columns=['Neighbourhood'])
tp_venues2.drop_duplicates(inplace=True) 

u_venues = tp_venues2.v_name.unique()

In [ ]:
u_venues_df = pd.DataFrame(u_venues)
u_venues_df.to_csv('u_venues_df.csv', encoding='utf_8_sig') 
files.download('u_venues_df.csv')

In [ ]:
def add_venues(): 

  paragraph = """<p>{}</p>""".format

  for lat, lng, name, cat in zip(tp_venues2['v_latitude'], tp_venues2['v_longitude'], tp_venues2['v_name'], tp_venues2['v_cat']):
    if cat == 'primary_school' or cat == 'secondary_school':
      color = 'blue'
      icon = 'pencil'
    elif cat == 'library': 
      color = 'green'
      icon = 'book'
    else: 
      color = 'gray'
      icon = 'home'
    label = '{}, {}'.format(name, cat)
    iframe = IFrame(html = paragraph(utf2asc(label)), width = 200, height = 100)
    popup = folium.Popup(iframe)
    folium.Marker([lat, lng], popup = popup,
                  icon = folium.Icon(icon = icon, color = color),                
                  ).add_to(map)  

  return map

In [ ]:
# Check neighborhoods where venues really locate

venue_loc = []

for id, venue, lat, lng in zip(tp_venues2['v_id'], tp_venues2['v_name'], tp_venues2['v_latitude'], tp_venues2['v_longitude']):
  point = Point(lng, lat)
  for feature in tp['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        venue_loc.append([id, venue, feature['properties']['TVNAME']])

venue_loc = pd.DataFrame(venue_loc, columns = ['v_id', 'v_name', 'venue_neigh']) 
  
venue_loc

In [ ]:
tp_venues3 = pd.merge(tp_venues2, venue_loc, on = 'v_id')

In [ ]:
# unique education resources located in Taipei City
tp_venues3.shape

In [ ]:
# Check the numbers of education resources located in each neighborhood

tp_edu2 = tp_venues3.groupby('venue_neigh')['v_cat'].value_counts().unstack(fill_value=0)
tp_edu2 = tp_edu2.reset_index()
tp_edu2['Total'] = tp_edu2['library'] + tp_edu2['museum'] + tp_edu2['primary_school'] + tp_edu2['secondary_school']
tp_edu2.sort_values('Total', ascending=False)

# Start Clustering

In [ ]:
tp_edu1

## Clustered by numbers of different education resources 

In [ ]:
edu_clustering = tp_edu1.loc[:, ['Library', 'Museum', 'School']]
edu_clustering

In [ ]:
kclusters = 5
kmeans = KMeans(n_clusters = kclusters, random_state=0)

In [ ]:
kmeans.fit(edu_clustering)

In [ ]:
edu_clustering.insert(0, 'Cluster1', kmeans.labels_)

edu_clustering

In [ ]:
dist_neigh = pd.DataFrame(tp_edu1['dist_neigh'])
dist_neigh

In [ ]:
result = pd.concat([dist_neigh, edu_clustering], axis=1)
result

In [ ]:
cluster_stat = edu_clustering.groupby('Cluster1').agg(['mean', 'count'])
cluster_stat

In [ ]:
edu_clustering.loc[edu_clustering['Cluster1'] == 1, 'Cluster1_reassigned'] = 4
edu_clustering.loc[edu_clustering['Cluster1'] == 4, 'Cluster1_reassigned'] = 3 
edu_clustering.loc[edu_clustering['Cluster1'] == 3, 'Cluster1_reassigned'] = 2 
edu_clustering.loc[edu_clustering['Cluster1'] == 0, 'Cluster1_reassigned'] = 1 
edu_clustering.loc[edu_clustering['Cluster1'] == 2, 'Cluster1_reassigned'] = 0 
edu_clustering

In [ ]:
edu_clustering.drop('Cluster1', axis=1, inplace=True)
edu_clustering.groupby('Cluster1_reassigned').agg(['mean', 'count'])

In [ ]:
tp_edu1.insert(0, 'Cluster1', edu_clustering['Cluster1_reassigned'])

tp_edu1

In [ ]:
# Show clusters on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'Accessible Education',
    data = tp_edu1,
    columns = ['dist_neigh', 'Cluster1'],
    key_on = 'properties.TVNAME',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Education Resources').add_to(map)

map

## Clustered by numbers of education resources per capita under 15

In [ ]:
edu_clustering1 = tp_edu1.loc[:, ['Library', 'Museum', 'School']]
edu_clustering1 = edu_clustering1.div(tp_edu1['under_15'], axis=0)
edu_clustering1 = 1000 * edu_clustering1
edu_clustering1

In [ ]:
edu_clustering1.shape

In [ ]:
kmeans.fit(edu_clustering1)

In [ ]:
edu_clustering1.insert(0, 'Cluster2', kmeans.labels_)

edu_clustering1

In [ ]:
result = pd.concat([dist_neigh, edu_clustering1], axis=1)
result

In [ ]:
cluster_stat1 = edu_clustering1.groupby('Cluster2').agg(['mean', 'count'])
cluster_stat1

In [ ]:
edu_clustering1.loc[edu_clustering1['Cluster2'] == 4, 'Cluster2_reassigned'] = 4
edu_clustering1.loc[edu_clustering1['Cluster2'] == 2, 'Cluster2_reassigned'] = 3 
edu_clustering1.loc[edu_clustering1['Cluster2'] == 0, 'Cluster2_reassigned'] = 2 
edu_clustering1.loc[edu_clustering1['Cluster2'] == 1, 'Cluster2_reassigned'] = 1 
edu_clustering1.loc[edu_clustering1['Cluster2'] == 3, 'Cluster2_reassigned'] = 0 
edu_clustering1

In [ ]:
cluster_stat1 = edu_clustering1.groupby('Cluster2_reassigned')['Library', 'Museum', 'School'].mean()
cluster_stat1

In [ ]:
edu_clustering1.drop('Cluster2', axis=1, inplace=True)
edu_clustering1.groupby('Cluster2_reassigned').agg(['mean', 'count'])

In [ ]:
tp_edu1.insert(0, 'Cluster2', edu_clustering1['Cluster2_reassigned'])

tp_edu1.head()

In [ ]:
# Show clusters on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'Accessible Education',
    data = tp_edu1,
    columns = ['dist_neigh', 'Cluster2'],
    key_on = 'properties.TVNAME',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Education Resources').add_to(map)

map

## Clustered by education resources per capita and household income

In [ ]:
edu_clustering2 = tp_edu1.loc[:, ['edu_Total', 'under_15', 'income_median']]
edu_clustering2['edu_per_cap'] = 1000 * edu_clustering2['edu_Total'].div(edu_clustering2['under_15'], axis = 0)
edu_clustering2.drop(['edu_Total', 'under_15'], axis = 1, inplace =True)

edu_clustering2

In [ ]:
result = pd.concat([dist_neigh, edu_clustering2], axis=1)
result

In [ ]:
edu_clustering2.describe()

In [ ]:
normalized_df = (edu_clustering2 - edu_clustering2.min()) / (edu_clustering2.max() - edu_clustering2.min())
normalized_df

In [ ]:
kmeans.fit(normalized_df)

In [ ]:
edu_clustering2.insert(0, 'Cluster3', kmeans.labels_)

edu_clustering2

In [ ]:
cluster_stat2 = edu_clustering2.groupby('Cluster3').agg(['mean', 'count'])
cluster_stat2

In [ ]:
normalized_df.insert(0, 'Cluster3', kmeans.labels_)

normalized_df

In [ ]:
cluster_stat2 = normalized_df.groupby('Cluster3')['income_median', 'edu_per_cap'].mean()
cluster_stat2

In [ ]:
normalized_df.loc[normalized_df['Cluster3'] == 2, 'Cluster3_reassigned'] = 4
normalized_df.loc[normalized_df['Cluster3'] == 1, 'Cluster3_reassigned'] = 3 
normalized_df.loc[normalized_df['Cluster3'] == 3, 'Cluster3_reassigned'] = 2 
normalized_df.loc[normalized_df['Cluster3'] == 4, 'Cluster3_reassigned'] = 1 
normalized_df.loc[normalized_df['Cluster3'] == 0, 'Cluster3_reassigned'] = 0 
normalized_df

In [ ]:
edu_clustering2.insert(0, 'Cluster3_reassigned', normalized_df['Cluster3_reassigned'])
edu_clustering2

In [ ]:
result = pd.concat([dist_neigh, edu_clustering2], axis=1)
result

In [ ]:
cluster_stat2 = edu_clustering2.groupby('Cluster3_reassigned').agg(['mean', 'count'])
cluster_stat2

In [ ]:
tp_edu1.insert(0, 'Cluster3', normalized_df['Cluster3_reassigned'])

tp_edu1.head()

In [ ]:
# Show clusters on map

map = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data = tp,
    name = 'Accessible Education',
    data = tp_edu1,
    columns = ['dist_neigh', 'Cluster3'],
    key_on = 'properties.TVNAME',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Education Resources').add_to(map)

map

In [ ]:
add_label()
add_venues()
map